### 1. 데이터 준비

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class ASL_Dataset(Dataset):
  def __init__(self, path):
    super(ASL_Dataset, self).__init__()

    df = pd.read_csv(path)
    self.y = df['label']
    del df['label']
    self.x = df.values



  def __getitem__(self, index):
    x_sample = np.uint8( self.x[index].reshape(28,28) )
    x_tensor = torch.tensor ( x_sample[np.newaxis,:,:] ).float()
    y_tensor = torch.tensor ( self.y[index] )
    return x_tensor, y_tensor



  def __len__(self):
    return self.x.shape[0]

In [7]:
train_data = ASL_Dataset("sign_mnist_train.csv")
valid_data = ASL_Dataset("sign_mnist_valid.csv")

train_loader = DataLoader(dataset=train_data, batch_size=1000, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=1000, shuffle=False)

### 2. 뉴럴네트워크 모델링

In [19]:
import torch.nn as nn
class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
      self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
      self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
      self.relu = nn.ReLU()
      self.pool = nn.MaxPool2d(2,2)
      self.fc = nn.Linear(7*7*32, 24)

  def forward(self, x):
    x = self.relu( self.conv1(x) )
    x = self.pool(x)   # 14X14X16
    x = self.relu( self.conv2(x) )
    x = self.pool(x)   # 7X7X32
    x = self.relu( self.conv3(x) )   # 7x7x32

    x = x.view(-1, 7*7*32)
    x = self.fc(x)
    return x

In [20]:
convnet = Net()

### 3.손실함수와 최적화함수의 지정

In [23]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

### 4. 뉴럴네트워크 학습

In [24]:
for epoch in range(5):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()

        inputs, labels = data
        pred = convnet(inputs)

        loss = loss_function(pred, labels)

        loss.backward()

        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, " , Loss:", loss_val)

Epoch: 1  , Loss: 139.78275990486145
Epoch: 2  , Loss: 41.181666135787964
Epoch: 3  , Loss: 12.290689006447792
Epoch: 4  , Loss: 4.458873055875301
Epoch: 5  , Loss: 1.911327749490738


### 5. 뉴럴네트워크 평가

In [26]:
pred_list = []
label_list = []
for itr, data in enumerate(valid_loader):
  inputs, labels = data
  pred_test = convnet(inputs)
  pred_category = torch.argmax(pred_test, dim=1)

  pred_list = pred_list + list(pred_category)
  label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list) ==np.array(label_list) )
print("Test accuracy: ", accu)

Test accuracy:  0.8623814835471277
